<h2> A1: Regression Model Development</h2>
<h3> ~ Bindushree R P</h3>

In the steps below, we import all the packages and create models to predict Birth Weight with the best suitable variables.
The variables were split into continuous, interval/count and categorical as specified below:
<br>
Continuous variables: mage, meduc, fage, feduc, cigs, drink
<br>
Count/Interval variables: npvis, monpre
<br>
Categorical variables: male, mwhte, mblck, moth, fwhte, fblck, foth
<br>
Target Variable: bwght
<br>
All the models were developed and the best model was marked with the asteric in the final model comparison output.
<br>
Source: Used regression model code from the classroom scripts 3 and 4.
<br>
Final model and result: Final model was selected based on the highest test score and less than 0.05 difference. 
The final results were displayed in the form of a dynamic string at the end of the code.

In [ ]:
# importing all the needed libraries: data science, graphical output, mathematical essentials and regression modeling
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import statsmodels.formula.api as smf 


#setting print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#loading the file
file = "./__datasets/birthweight_low_A1.xlsx"


#reading the file 
df_birthweight = pd.read_excel(io  = file,
                               sheet_name = 0,
                               header     = 0)


#outputting the first ten rows of the dataset
df_birthweight.head(n = 10)

In [ ]:
#printing the information of the dataframe to check the datatypes and number of total columns
df_birthweight.info()

In [ ]:
#looking at the number of missing values
df_birthweight.isnull().sum(axis=0)

In [ ]:
#Creating a list with continuous data
continuous_data = ['bwght','mage','meduc','fage','feduc','cigs','drink','male']

#developing a correlation matrix based on the list: continuous_data
df_birthweight_corr = df_birthweight[continuous_data].corr(method="pearson")

#filtering the results to only show the correlation of the continuous data with birth weight
df_birthweight_corr.loc[ : , "bwght"].round(decimals = 2).sort_values(ascending = False)

In [ ]:
#defining and instantiating a model object for regression
lm_best = smf.ols(formula = """bwght ~ fage + mage + cigs + drink + male
                               + feduc + meduc""",
                  data = df_birthweight)


#fitting the data into the model
results = lm_best.fit()


#analysing the summary output
print(results.summary())

#From this output, it can be seen that the p values for mothers education, fathers education and baby being male 
#are very high. We shall try to feature engineer some variables and check how the R square and P values vary.

In [ ]:
#creating a copy of the data frame to translate and impute missing values going forward
df_birthweight_translated = pd.DataFrame.copy(df_birthweight)

#checking for the number of missing values in our translated file
df_birthweight_translated.isnull().sum(axis=0)

In [ ]:
# looping to detect features with missing values
for col in df_birthweight_translated:

    # creating columns for missing valued columns and imputing with 1s if missing and 0 if not
    if df_birthweight_translated[col].isnull().astype(int).sum() > 0:
        df_birthweight_translated['m_'+col] = df_birthweight_translated[col].isnull().astype(int)

# summing the missing value flags to check the results of the loop above
df_birthweight_translated[    ['m_meduc', 'm_npvis','m_feduc']    ].sum(axis = 0)

In [ ]:
# imputing the missing values with median for meduc, npvis and feduc

for col in df_birthweight_translated.columns:
    fill = df_birthweight_translated.median()
    df_birthweight_translated.fillna(value = fill,
                                    inplace = True)

#df_birthweight_translated.isnull().sum(axis=0)

In [ ]:
#INSTANTIATE a model object
lm_best = smf.ols(formula = """bwght ~ fage + mage + cigs + drink + meduc + feduc 
                               + m_meduc + m_feduc + male + npvis + m_npvis""",
                  data = df_birthweight_translated)


#FIT the data into the model object
results = lm_best.fit()


#analyze the SUMMARY output
print(results.summary())

In [ ]:
#checking the results of correlation of all continuous variables with birthweight

continuous_data = ['bwght', 'mage','meduc','fage','feduc','cigs','drink']

df_birthweight_corr = df_birthweight_translated[continuous_data].corr(method = 'pearson')
df_birthweight_corr.loc[ : , "bwght"].round(decimals = 2).sort_values(ascending = False)

In [ ]:
# log transforming variables and saving it to the dataset as columns
df_birthweight_translated['log_bwght'] = np.log(df_birthweight_translated['bwght'])
df_birthweight_translated['log_mage'] = np.log(df_birthweight_translated['mage'])
df_birthweight_translated['mage_multiply_fage'] = df_birthweight_translated['mage'] * df_birthweight_translated['fage']

In [ ]:
#Checking the correlation with the log transformed variables
#INSTANTIATE a model object
lm_best = smf.ols(formula = """log_bwght ~ fage + cigs + drink + meduc + feduc + 
                               mblck + fblck + male + m_npvis + mage_multiply_fage""",
                  data = df_birthweight_translated)


#FIT the data into the model object
results = lm_best.fit()


#analyze the SUMMARY output
print(results.summary())

In [ ]:
#importing all the stats model and scikit packages needed
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.linear_model

In [ ]:
#preparing data with explanatory columns by dropping other columns
birthweight_data   = df_birthweight_translated.drop(["bwght",
                                                     "log_bwght",
                                                     "omaps",
                                                     "fmaps",
                                                     "log_mage",
                                                     "mage_multiply_fage"],
                                                      axis = 1)


#Creating target variables for regression
birthweight_target = df_birthweight_translated.loc[ : , "bwght"]
log_birthweight_target = df_birthweight_translated.loc[ : , "log_bwght"]


#training and testing sets with test size of 0.25 and random state of 219
x_train, x_test, y_train, y_test = train_test_split(
            birthweight_data,
            birthweight_target,
            test_size = 0.25,
            random_state = 219)


#checking the shapes of the datasets
print(f"""
Training Data
-------------
X-side: {x_train.shape}
y-side: {y_train.shape}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape}
""")

In [ ]:
#declaring set of x-variables
x_variables = ['meduc', 'drink', 'cigs', 'fage']


# looping to make x-variables suitable for statsmodels
for val in x_variables:
    print(f"{val} +")

In [ ]:
#merging X_train and y_train to use in all the stats models
birthweight_train = pd.concat([x_train, y_train], axis = 1)


#building the model
lm_best = smf.ols(formula =  """bwght ~ meduc +
                                        drink +
                                        cigs +
                                        fage""",
                                data = birthweight_train)


#fitting the model based on our data
results = lm_best.fit()



#analyzing the summary output
print(results.summary())

In [ ]:
# applying modelin scikit-learn

#preparing ols model to use the x-variables defined earlier
ols_data = df_birthweight_translated.loc[ : , x_variables]


#preparing the response data for the model
birthweight_target = df_birthweight_translated.loc[ : , "bwght"]


#Creating two train test split sets for using in the models
#all variables data is used to create FULL train test split 
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            birthweight_data,     
            birthweight_target,
            test_size = 0.25,
            random_state = 219)


#only x-variables defined withing OLS data to build OLS test train split
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(
            ols_data,
            birthweight_target, 
            test_size = 0.25,
            random_state = 219)


In [ ]:
#model object is instantiated
lr = LinearRegression()


#fitting the model to training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)


#predicting on new dataset
lr_pred = lr_fit.predict(x_test_OLS)


#scoring the results to get the R square for test and train
print('OLS Training Score :', lr.score(x_train_OLS, y_train_OLS).round(4))
print('OLS Testing Score  :',  lr.score(x_test_OLS, y_test_OLS).round(4))

lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4)
lr_test_score  = lr.score(x_test_OLS, y_test_OLS).round(4)

#displaying and saving the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

In [ ]:
#feature name is zipped into its coefficient 
lr_model_values = zip(birthweight_data[x_variables].columns,
                      lr_fit.coef_.round(decimals = 2))


#storing model features in a placeholder list
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]


#printing the feature coefficient pair
for val in lr_model_values:
    lr_model_lst.append(val)
    

#printing the results along with the coefficient name
for pair in lr_model_lst:
    print(pair)

In [ ]:
#model object is instantiated
lasso_model = sklearn.linear_model.Lasso(alpha = 1.0,
                                         normalize = True) # default magitude


#fitting the model to training data
lasso_fit = lasso_model.fit(x_train_OLS, y_train_OLS)


#predicting on new dataset
lasso_pred = lasso_fit.predict(x_test_OLS)


#scoring the results to get the R square for test and train
print('Lasso Training Score :', lasso_model.score(x_train_OLS, y_train_OLS).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test_OLS, y_test_OLS).round(4))


#using the below code given by Prof. Chase to save the scores and print the resulting train and test model scores

# saving scoring data for future use
lasso_train_score = lasso_model.score(x_train_OLS, y_train_OLS).round(4)
lasso_test_score  = lasso_model.score(x_test_OLS, y_test_OLS).round(4)


#displaying and saving the gap between training and testing
print('Lasso Train-Test Gap :', abs(lasso_train_score - lasso_test_score).round(4))
lasso_test_gap = abs(lasso_train_score - lasso_test_score).round(4)

In [ ]:
#feature name is zipped into its coefficient along with the used data for this model
lasso_model_values = zip(ols_data, lasso_fit.coef_.round(decimals = 2))

#storing model features in a placeholder list
lasso_model_lst = [('intercept', lasso_fit.intercept_.round(decimals = 2))]


#printing the feature coefficient pair
for val in lasso_model_values:
    lasso_model_lst.append(val)
    
#printing the results along with the coefficient name
for pair in lasso_model_lst:
    print(pair)

In [ ]:
#printing the feature coefficient pair
for feature, coefficient in lasso_model_lst:
        
        if coefficient == 0:
            lasso_model_lst.remove((feature, coefficient))

#printing the results along with the coefficient name
for pair in lasso_model_lst:
    print(pair)

In [ ]:
#model object is instantiated
ard_model = sklearn.linear_model.ARDRegression()


#fitting the model to training data
ard_fit = ard_model.fit(x_train_OLS,y_train_OLS)


#predicting on new dataset
ard_pred = ard_fit.predict(x_test_OLS)


print('Training Score:', ard_model.score(x_train_OLS, y_train_OLS).round(4))
print('Testing Score :', ard_model.score(x_test_OLS, y_test_OLS).round(4))


#saving scoring data for future use
ard_train_score = ard_model.score(x_train_OLS, y_train_OLS).round(4)
ard_test_score  = ard_model.score(x_test_OLS, y_test_OLS).round(4)


#displaying and saving the gap between training and testing
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)

In [ ]:
#feature name is zipped into its coefficient along with the used data for this model
ard_model_values = zip(birthweight_data[x_variables], ard_fit.coef_.round(decimals = 5))

#storing model features in a placeholder list
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


#printing the feature coefficient pair
for val in ard_model_values:
    ard_model_lst.append(val)
    

#printing the results along with the coefficient name
for pair in ard_model_lst:
    print(pair)

In [ ]:
#Source: Prof.Chase' classroom script
#dropping coefficients that are equal to zero

#printing out each feature-coefficient pair one by one
for feature, coefficient in ard_model_lst:
        
        if coefficient == 0:
            ard_model_lst.remove((feature, coefficient))

            
#checking the results
for pair in ard_model_lst:
    print(pair)

Building model using KNN regression method to check if the model yields best results.


In [ ]:
#KNN and standard scaler are imported 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

#StandardScaler() object is instantiated
scaler = StandardScaler()


#fitting the scaler with birthweight data
scaler.fit(birthweight_data)


#transforming data after fit
x_scaled = scaler.transform(birthweight_data)


#converting scaled data into a DataFrame
x_scaled_df = pd.DataFrame(x_scaled)


# checking the results
x_scaled_df.describe().round(2)

In [ ]:
# adding labels to the scaled DataFrame
x_scaled_df.columns = birthweight_data.columns

#  Checking pre- and post-scaling of the data
print(f"""
Dataset BEFORE Scaling
----------------------
{np.var(birthweight_data)}


Dataset AFTER Scaling
----------------------
{np.var(x_scaled_df)}
""")

In [ ]:
# Unscaled Dataset

# subsetting the original dataset
birthweight_subset = birthweight_data.loc[ : , ['meduc', 'drink', 'cigs', 'fage']]


# UNSCALED correlation matrix
df_corr = birthweight_subset.corr().round(2)


# setting figure size and plot window
fig, ax = plt.subplots(figsize = (16, 16))
plt.subplot(1, 2, 1)


# heatmap of UNSCALED correlations
sns.heatmap(df_corr,
            cmap = 'twilight',
            square = True,
            annot = True,
            cbar = False,
            linecolor  = 'black', 
            linewidths = 0.5)


# Scaled Dataset

# SCALED correlation matrix
df_scaled_corr = x_scaled_df.loc[ : , ['meduc', 'drink', 'cigs', 'fage']].corr().round(2)


# titling the plot
plt.title("BEFORE Standardization")


# setting plot window
plt.subplot(1, 2, 2)


# heatmap of SCALED correlations
sns.heatmap(df_scaled_corr,
            cmap = 'twilight',
            square = True,
            annot = True,
            cbar = False,
            linecolor  = 'black',
            linewidths = 0.5)


# titling the plot
plt.title("AFTER Standardization")
plt.savefig('./__analysis_images/Corrplots Before and After Scaling_birthweight.png')
plt.show()

In [ ]:
# this is the exact code we were using before
x_train, x_test, y_train, y_test = train_test_split(
            x_scaled_df,
            birthweight_target,
            test_size = 0.25,
            random_state = 219)

In [ ]:
# INSTANTIATING a KNN model object
knn_reg = KNeighborsRegressor(algorithm = 'auto',
                              n_neighbors = 5)


# FITTING to the training data
knn_fit = knn_reg.fit(x_train, y_train)


# PREDICTING on new data
knn_reg_pred = knn_fit.predict(x_test)


# SCORING the results
print('KNN Training Score:', knn_reg.score(x_train, y_train).round(4))
print('KNN Testing Score :', knn_reg.score(x_test, y_test).round(4))


# saving scoring data for future use
knn_reg_score_train = knn_reg.score(x_train, y_train).round(4)
knn_reg_score_test  = knn_reg.score(x_test, y_test).round(4)


# displaying and saving the gap between training and testing
print('KNN Train-Test Gap:', abs(knn_reg_score_train - knn_reg_score_test).round(4))
knn_reg_test_gap = abs(knn_reg_score_train - knn_reg_score_test).round(4)

In [ ]:
# creating lists for training set accuracy and test set accuracy
training_accuracy = []
test_accuracy     = []


# building a visualization of 1 to 50 neighbors
neighbors_settings = range(1, 51)


for n_neighbors in neighbors_settings:
    # Building the model
    clf = KNeighborsRegressor(n_neighbors = n_neighbors)
    clf.fit(x_train, y_train)
    
    # Recording the training set accuracy
    training_accuracy.append(clf.score(x_train, y_train))
    
    # Recording the generalization accuracy
    test_accuracy.append(clf.score(x_test, y_test))


# plotting the visualization
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()
plt.show()

In [ ]:
# finding the optimal number of neighbors
opt_neighbors = test_accuracy.index(max(test_accuracy)) + 1
print(f"""The optimal number of neighbors is {opt_neighbors}""")

In [ ]:
#Saving all the birthweight prediction into an excel
prediction_results = pd.DataFrame(data = {
    'Original Birthweight' : y_test_FULL,
    'LR Predictions'       : lr_pred.round(decimals = 2),
    'Lasso Predictions'    : lasso_pred.round(decimals = 2),
    'ARD Predictions'      : ard_pred.round(decimals = 2),
    'LR Deviation'         : lr_pred.round(decimals = 2) - y_test_FULL,
    'Lasso Deviation'      : lasso_pred.round(decimals = 2) - y_test_FULL,
    'ARD Deviation'        : ard_pred.round(decimals = 2) - y_test_FULL,
    })


prediction_results.to_excel(excel_writer = './__model_results/linear_model_predictions_birthweight.xlsx',
                            index = False)

In [ ]:
#comparing model results

print(f"""
Model      Train Score      Test Score       Difference
-----      -----------      ----------       ----------
OLS        {lr_train_score}           {lr_test_score}            {lr_test_gap}
Lasso      {lasso_train_score}           {lasso_test_score}            {lasso_test_gap}
**ARD**    {ard_train_score}           {ard_test_score}            {ard_test_gap}
""")


#creating a dictionary for model results
model_performance = {
    
    'Model Type'    : ['OLS', 'Lasso', 'ARD'],
           
    'Training' : [lr_train_score, lasso_train_score,
                                   ard_train_score],
           
    'Testing'  : [lr_test_score, lasso_test_score,
                                   ard_test_score],
                    
    'Train-Test Gap' : [lr_test_gap, lasso_test_gap,
                                        ard_test_gap],
                    
    'Model Size' : [len(lr_model_lst), len(lasso_model_lst),
                                    len(ard_model_lst)],
                    
    'Model' : [lr_model_lst, lasso_model_lst, ard_model_lst]}


#converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)


#sending model results to Excel
model_performance.to_excel('./__model_results/linear_model_performance_birthweight.xlsx',
                           index = False)

print(f"""ARD had the highest test score and hence my final model.""")